In [2]:
import tabula
import numpy as np
import pandas as pd
import os
import re
import math

In [4]:
relative_path_pdf = "../HTML/Output_html_Chapter13/kaggle/working/out/pages/"
relative_dictionary_path = "../Data/Chapter13_Data.csv"
relative_result_path = "../HTML/Output_html_Chapter13/Chapter13_Data.csv"
# Get the absolute path by joining the current directory with the relative path
absolute_path_pdf = os.path.join(os.getcwd(), relative_path_pdf)
absolute_dictionary_path = os.path.join(os.getcwd(), relative_dictionary_path)
absolute_result_path = os.path.join(os.getcwd(), relative_result_path)

In [5]:
os.listdir(absolute_path_pdf)

['page_1.pdf',
 'page_2.pdf',
 'page_3.pdf',
 'page_5.pdf',
 'page_6.pdf',
 'page_4.pdf']

In [6]:
output_list = []

In [7]:
# Get a list of PDF files sorted by filename
pdf_files = sorted([file for file in os.listdir(absolute_path_pdf) if file.endswith(".pdf")], key=lambda x: [int(s) if s.isdigit() else s.lower() for s in re.split('(\d+)', x)])

# Iterate through all PDF files
for file in pdf_files:
    file_path = os.path.join(absolute_path_pdf, file)
    dfs = tabula.read_pdf(file_path, pages='all', stream=True)
    if dfs:
        output_list.append(dfs[0])  # Assuming you want to append the first DataFrame from each PDF

In [8]:
len(output_list)

6

In [9]:
output_list[3]

,Unnamed: 0,Unnamed: 1,ಅಂಗನಾ ೇಂದಗಳ / Anganawadi Centres,Unnamed: 2
0,Sl,NaN,ಸಂತ ಕಟಡ ಾಾ ಕಟಡ ಇತೆ,ಒಟು % of
1,No,Districts,NaN,Anganawadis
2,NaN,NaN,Own School Others,Total
3,NaN,NaN,NaN,having Own
4,NaN,NaN,Building Building,Buildings
5,1,2,1356 1357 1358,1359 1360
6,1,Belagavi,3127 301 1898,5326 58.71
7,2,Bagalkot,1428 111 682,2221 64.30
8,3,Vijayapura,1313 120 880,2313 56.77
9,4,Kalburgi,1969 117 1044,3130 62.91


In [10]:
#adding header
#new_row = [1, 2,  np.nan,967,968,969,970,971,972,973,974,975,976]
#index = 0
#output_list[3].loc[index + 1:] = output_list[3].loc[index:].shift(1)
#output_list[3].loc[index] = new_row
#output_list[3]

def add_header(df_list,idx_list, new_list):
    index = 0
    for i in range(len(idx_list)):
        df_list[idx_list[i]].loc[index + 1:] = df_list[idx_list[i]].loc[index:].shift(1)
        df_list[idx_list[i]].loc[index] = new_list[i]
    return df_list
            

In [11]:
def find_last_index(df_col):
    pattern = r'[-+]?\b\d+\b(?:\.\d+)?'
    for idx in range(len(df_col)):
            if isinstance(df_col.iloc[idx], str):
                if re.findall(pattern, df_col.iloc[idx]):
                    numbers_1 = re.findall(pattern, df_col.iloc[idx])
                    numbers_1 = [int(num) if '.' not in num else float(num) for num in numbers_1]
                    #print(numbers_1)
                    if numbers_1 and numbers_1[0] == 30:
                        return idx+1
   
    return (len(df_col) - 1)

In [12]:
def find_header(df_list):
    # Initialize variables to track the start and end indices
    index_list = []
    drop_index =  []
    i=0
    for df in df_list:
        df_col = df.iloc[:,0]
        df_col = df_col.astype(str)

        numbers_1 = []
        numbers_2 = []
        # Regex pattern to match numeric values
        #pattern = r'\b\d+\b'
        pattern = r'[-+]?\b\d+\b(?:\.\d+)?'
        flag = 0
        for idx in range(len(df_col) - 1):
            #print(idx, df_col.iloc[idx])
            if isinstance(df_col.iloc[idx], str):
                if re.findall(pattern, df_col.iloc[idx]):
                    numbers_1 = re.findall(pattern, df_col.iloc[idx])
                    numbers_1 = [int(num) if '.' not in num else float(num) for num in numbers_1]
                    #print(numbers_1)
            if isinstance(df_col.iloc[idx+1], str):
                if re.findall(pattern, df_col.iloc[idx+1]):
                    numbers_2 = re.findall(pattern, df_col.iloc[idx+1])
                    numbers_2 = [int(num) if '.' not in num else float(num) for num in numbers_2]
                    #print(numbers_2)
            if numbers_1 and numbers_2 and numbers_1[0] == 1 and numbers_2[0] == 1:
                start_index = idx
                end_index = find_last_index(df_col)
                index_list.append([start_index, end_index])
                flag=1
                i += 1
                break

        if flag == 0:
            drop_index.append(i)
            i +=1
            
    return index_list, drop_index
            

In [13]:
index_list, drop_list = find_header(output_list)

In [14]:
drop_list

[]

In [15]:
index_list

[[9, 40], [12, 43], [8, 39], [5, 36], [11, 42], [10, 41]]

In [16]:
len(index_list)

6

In [17]:
def remove_first_col(df_list, drop_index):
    new_df_list = []
    if len(drop_list) > 0:
        for idx in range(len(df_list)):
            if idx in drop_list:
                df = df_list[idx]
                df = df.drop(df.columns[0],axis=1)
                new_df_list.append(df)
            else:
                new_df_list.append(df_list[idx])
        return new_df_list
    return df_list

In [18]:
def get_final_index(df_list, index_ll, drop_index):
    df = df_list
    while len(drop_index) > 0:
        df = remove_first_col(df, drop_index)
        index_ll, drop_list = find_header(df)
        drop_index = drop_list
    return df, index_ll, drop_index

In [19]:
new_output_list, new_index_list, new_drop_list = get_final_index(output_list, index_list, drop_list)

In [20]:
new_index_list

[[9, 40], [12, 43], [8, 39], [5, 36], [11, 42], [10, 41]]

In [21]:
new_output_list[1]

,Unnamed: 0,13.2 ಾರಂಭಂದ 31-3-2020ರ ವೆೆ ಾಂಂದ ಮತು ಸಾರಂದ ಪೆದ,ಾಲ ಮತು ಸುತು 
0,NaN,Loan and revolving fund availed from the Bank ...,NaN
1,NaN,ಸಂಗಹೆಾದ ಸಸಾಯ ಸಂಘಗಳ,NaN
2,NaN,ಒಟು ಉಾಯ Self Help Groups,NaN
3,Sl,Districts (ೋ ರೂ.ಗಳ),NaN
4,No,ಾಂಂದ ಾಲ ಪೆದಾಂಂದ ಾಲ ಪೆದ,ಸಾರಂದ ಪೆದ ಸುತ
5,NaN,ಸಸಾಯ ಸಂಘಗಳ ಒಟು ತ,
6,NaN,Total of (ೋ ರೂ.ಗಳ),NaN
7,NaN,Accumulated No. of SHGs availed,(ೋ ರೂ.ಗಳ)
8,NaN,savings Loan from Banks Total Loan availed from,Revolving Funds
9,NaN,(Rs. In Crores) Banks,received from Govt


In [22]:
def remove_header(df, ind_list):
    new_output_list=[]
    for idx in range(len(ind_list)):
        df_page = df[idx]
        #print(index_list[idx][0], index_list[idx][1])
        #print(df_page)
        df_new = df_page.drop(index=df_page.index[~df_page.index.isin(range(ind_list[idx][0], ind_list[idx][1]))])
        df_new.reset_index(drop=True, inplace=True)
        new_output_list.append(df_new)
    return new_output_list

In [23]:
corrected_list = remove_header(new_output_list, new_index_list)

In [24]:
corrected_list[3]

,Unnamed: 0,Unnamed: 1,ಅಂಗನಾ ೇಂದಗಳ / Anganawadi Centres,Unnamed: 2
0,1,2,1356 1357 1358,1359 1360
1,1,Belagavi,3127 301 1898,5326 58.71
2,2,Bagalkot,1428 111 682,2221 64.30
3,3,Vijayapura,1313 120 880,2313 56.77
4,4,Kalburgi,1969 117 1044,3130 62.91
5,5,Bidar,1259 93 541,1893 66.51
6,6,Raichur,1639 58 965,2662 61.57
7,7,Koppal,1286 58 506,1850 69.51
8,8,Gadag,865 32 269,1166 74.19
9,9,Dharwad,809 96 602,1507 53.68


In [25]:
len(corrected_list)

6

In [26]:
def check_nan(df_col):
    nan_columns = df_col[df_col.isna()].index.tolist()
    if len(nan_columns) == len(df_col):
        return []
    return nan_columns

In [27]:
def set_header_row(df_list_all):
    df_list = df_list_all
    new_df_list = []
    for df in df_list:
        header_row = df.iloc[0].tolist()
        for idx in range(len(header_row)):
            nan_col = check_nan(df.iloc[:,idx])
            if isinstance(header_row[idx], float) and math.isnan(header_row[idx]) and len(nan_col)>0 and len(nan_col)<len(df.iloc[:,idx]):
                j=idx+1
                while j<len(header_row) and isinstance(header_row[j], float) and math.isnan(header_row[j]):
                    j+=1
                if j<len(header_row) and not (isinstance(header_row[j], float) and math.isnan(header_row[j])):
                    words = header_row[j].split()
                    if len(words) > 1:
                        # Update the value of Column1 with the first word
                        header_row[idx] = words[0]
                        # Remove the first word from Column2
                        header_row[j] = ' '.join(words[1:])
                    elif len(words) == 1:
                        header_row[idx] = words[0]
                        header_row[j] = np.nan
            df.iloc[0] = header_row
        df.iloc[0] = header_row
        new_df_list.append(df)
    return new_df_list

In [28]:
corrected_header = set_header_row(corrected_list)

In [29]:
len(corrected_header)

6

In [30]:
corrected_header[3]

,Unnamed: 0,Unnamed: 1,ಅಂಗನಾ ೇಂದಗಳ / Anganawadi Centres,Unnamed: 2
0,1,2,1356 1357 1358,1359 1360
1,1,Belagavi,3127 301 1898,5326 58.71
2,2,Bagalkot,1428 111 682,2221 64.30
3,3,Vijayapura,1313 120 880,2313 56.77
4,4,Kalburgi,1969 117 1044,3130 62.91
5,5,Bidar,1259 93 541,1893 66.51
6,6,Raichur,1639 58 965,2662 61.57
7,7,Koppal,1286 58 506,1850 69.51
8,8,Gadag,865 32 269,1166 74.19
9,9,Dharwad,809 96 602,1507 53.68


In [31]:
def set_index_col(df_list):
    new_df_list = []
    for df in df_list:
        nan_col = check_nan(df.iloc[:,0])
        if len(df.index) > 31 and len(nan_col)>1:
            for idx in range(len(nan_col)-1):
                if nan_col[idx]+2 == nan_col[idx+1] and nan_col[idx]+1 not in nan_col:
                    for i in range(df.shape[1]):
                        p = nan_col[idx]
                        q = nan_col[idx]+1
                        r = nan_col[idx+1]
                        if i>0 and not (isinstance(df.iloc[p,i], float) and math.isnan(df.iloc[p,i])) and  not (isinstance(df.iloc[r,i], float) and math.isnan(df.iloc[r,i])):
                            if (isinstance(df.iloc[q,i], float) and math.isnan(df.iloc[q,i])):
                                df.iloc[q,i] = ' '.join([df.iloc[p,i], df.iloc[r,i]])
                            elif not (isinstance(df.iloc[q,i], float) and math.isnan(df.iloc[q,i])):
                                df.iloc[q,i] = ' '.join([df.iloc[p,i], df.iloc[r,i], df.iloc[q,i]])
                    idx+=1
            df = df.drop(nan_col)
        df.reset_index(drop=True)
        new_df_list.append(df)
    return new_df_list
        

In [32]:
corrected_index = set_index_col(corrected_header)

In [33]:
len(corrected_index)

6

In [34]:
corrected_index[3]

,Unnamed: 0,Unnamed: 1,ಅಂಗನಾ ೇಂದಗಳ / Anganawadi Centres,Unnamed: 2
0,1,2,1356 1357 1358,1359 1360
1,1,Belagavi,3127 301 1898,5326 58.71
2,2,Bagalkot,1428 111 682,2221 64.30
3,3,Vijayapura,1313 120 880,2313 56.77
4,4,Kalburgi,1969 117 1044,3130 62.91
5,5,Bidar,1259 93 541,1893 66.51
6,6,Raichur,1639 58 965,2662 61.57
7,7,Koppal,1286 58 506,1850 69.51
8,8,Gadag,865 32 269,1166 74.19
9,9,Dharwad,809 96 602,1507 53.68


In [35]:
def first_column_split(df_list):
    # Initialize variables to track the start and end indices
    new_df = []
        
    for df in df_list:
        df_col = df.iloc[:,0]
        #print(df_col)
        # Regex pattern to match numeric values
        #pattern = r'\b\d+\b'
        pattern = r'[-+]?\b\d+\b(?:\.\d+)?'
        
        if isinstance(df_col.iloc[0], str):
            if re.findall(pattern, df_col.iloc[0]):
                numbers = re.findall(pattern, df_col.iloc[0])
                numbers = [int(num) if '.' not in num else float(num) for num in numbers]
                #numbers = [int(x) for x in re.findall(pattern, df_col.iloc[0])]
                if len(numbers) == 1:
                    new_df.append(df)
                else:
                    pattern = r'(?<=\d)\s'

                    # Split the strings based on the pattern and expand into separate columns
                    split_data = df_col.str.split(pattern, n=1, expand=True)

                    # Rename the columns
                    split_data.columns = ['col1', 'col2']
                    
                    concated_df = pd.concat([split_data, df.iloc[:, 1:]], axis = 1)
                    new_df.append(concated_df)
        elif df_col.dtype == float:
            df.iloc[:,0] = df.iloc[:,0].astype(int).astype(str)
            new_df.append(df)
    return new_df

In [36]:
new_list = first_column_split(corrected_index)

In [37]:
len(new_list)

6

In [38]:
new_list[3]

,Unnamed: 0,Unnamed: 1,ಅಂಗನಾ ೇಂದಗಳ / Anganawadi Centres,Unnamed: 2
0,1,2,1356 1357 1358,1359 1360
1,1,Belagavi,3127 301 1898,5326 58.71
2,2,Bagalkot,1428 111 682,2221 64.30
3,3,Vijayapura,1313 120 880,2313 56.77
4,4,Kalburgi,1969 117 1044,3130 62.91
5,5,Bidar,1259 93 541,1893 66.51
6,6,Raichur,1639 58 965,2662 61.57
7,7,Koppal,1286 58 506,1850 69.51
8,8,Gadag,865 32 269,1166 74.19
9,9,Dharwad,809 96 602,1507 53.68


In [39]:
def process_cell(cell):
    if isinstance(cell, str):
        if '-' in cell:
            cell = cell.replace('-', ' Null ').strip()
            cell = ' '.join(cell.split())  # Remove excess white spaces
        else:
            cell = ' '.join(cell.split())  # Remove excess white spaces
    return cell

In [40]:
def replace_with_nan(df_list):
    new_df_list = []
    for df in df_list:
        new_df = df.map(process_cell)
        new_df_list.append(new_df)
    return new_df_list

In [41]:
#replaced_list = replace_with_nan(new_list)
replaced_list = new_list

In [42]:
replaced_list[3]

,Unnamed: 0,Unnamed: 1,ಅಂಗನಾ ೇಂದಗಳ / Anganawadi Centres,Unnamed: 2
0,1,2,1356 1357 1358,1359 1360
1,1,Belagavi,3127 301 1898,5326 58.71
2,2,Bagalkot,1428 111 682,2221 64.30
3,3,Vijayapura,1313 120 880,2313 56.77
4,4,Kalburgi,1969 117 1044,3130 62.91
5,5,Bidar,1259 93 541,1893 66.51
6,6,Raichur,1639 58 965,2662 61.57
7,7,Koppal,1286 58 506,1850 69.51
8,8,Gadag,865 32 269,1166 74.19
9,9,Dharwad,809 96 602,1507 53.68


In [43]:
len(replaced_list)

6

In [44]:
def all_column_split(df_list):
    new_df_list = []
        
    for df in df_list:
        new_df = pd.DataFrame()
        pattern = r'\b\d+\b'  # Updated pattern to match numbers only
        for colName, colData in df.items():
            if isinstance(colData.values[0], str):
                if re.findall(pattern, colData.values[0]):
                    numbers = [int(x) for x in re.findall(pattern, colData.values[0])]
                    if len(numbers) == 1:
                        new_df = pd.concat([new_df, colData], axis=1)
                    elif len(numbers) == 0 and (pd.isnull(colData.values[0]) or pd.isna(colData.values[0])):
                        print("Removed NaN col")
                    elif len(numbers) > 1:
                        colData = colData.str.replace(r'\s+', ' ', regex=True)
                        split_columns = colData.str.rsplit(' ', n=(len(numbers)-1), expand=True)
                        new_df = pd.concat([new_df, split_columns], axis=1)

        new_df_list.append(new_df)
    return new_df_list


In [45]:
separated_list = all_column_split(replaced_list)

In [46]:
len(separated_list)

6

In [47]:
separated_list[2]

,Unnamed: 0,0,1,2,0,1,Unnamed: 1
0,1,2,1351,1352,1353,1354,1355
1,1,ೆಳಾ,12419,247756,8.64,9833,219128
2,2,ಾಗಲೋೆ,1950,105364,3.68,3963,99943
3,3,ಜಯಪರ,5347,134616,4.70,8683,127319
4,4,ಕಲು,5828,147349,5.14,4960,147008
5,5,ೕದ,4501,88464,3.09,5418,88464
6,6,ಾಯಚೂರು,2086,120613,4.21,6204,120613
7,7,ೊಪಳ,5444,78831,2.75,2568,74907
8,8,ಗದ,3167,72346,2.52,5629,37181
9,9,ಾರಾಡ,5119,100484,3.51,0,93521


In [48]:
def check_single_column(df_list):
    new_df_list = []
    for df in df_list:
        df.fillna(value="", inplace=True)
        num_col = len(df.columns)
        for idx in range(num_col-1, 1, -1):
            cardinality = len(str(df.iloc[0,idx]).split())
            #print("idx", idx, df.iloc[0,idx], cardinality)
            for i in range(len(df.iloc[:,idx])):
                if i>0:
                    cell_car = len(str(df.iloc[i,idx]).split())
                    #print("i", i, df.iloc[i,idx], cell_car)
                    if cell_car > cardinality:
                       
                        words_col1 = str(df.iloc[i, idx]).split()

                        # Extract the words from the second column
                        words_col2 = str(df.iloc[i, idx - 1]).split()
                        #print("more", words_col1, words_col2)
                        # Check if there are words in the first column
                        if words_col1:
                            # Move the last word of the first column to become the first word of the second column
                            if words_col2:
                                df.iloc[i, idx-1] = ' '.join(words_col2) + ' ' + words_col1[0]
                            else:
                                df.iloc[i, idx-1] = words_col1[0]

                            # Update the first column with the remaining words
                            df.iloc[i, idx] = ' '.join(words_col1[1:])
                        
                    if cell_car < cardinality:
                        
                        words_col1 = str(df.iloc[i, idx]).split()

                        # Extract the words from the second column
                        words_col2 = str(df.iloc[i, idx - 1]).split()
                        #print("less", words_col1, words_col2)
                        # Check if there are words in the first column
                        if words_col2:
                            # Move the last word of the first column to become the first word of the second column
                            if words_col1:
                                df.iloc[i, idx] = words_col2[-1] + ' ' + ' '.join(words_col1)
                            else:
                                df.iloc[i, idx] = words_col2[-1]

                            # Update the first column with the remaining words
                            df.iloc[i, idx-1] = ' '.join(words_col2[:-1])
                        if df.iloc[i, idx-1]==' ':
                            df.iloc[i, idx-1] = ''
                        
        new_df_list.append(df)
    return new_df_list

In [49]:
updated_list = check_single_column(separated_list)

In [50]:
len(updated_list)

6

In [51]:
updated_list[0]

,col1,0,1,2,3,4,5,6
0,1,2,1341,1342,1343,1344,1345,1346
1,1,ೆಳಾ,16023,9.78,40242,13039,202300,255581
2,2,ಾಗಲೋೆ,4879,2.98,15892,7282,49554,72728
3,3,ಜಯಪರ,4708,2.87,14027,830,39060,53917
4,4,ಕಲು,7655,4.67,30228,1326,76536,108090
5,5,ೕದ,4051,2.47,14588,5986,22034,42608
6,6,ಾಯಚೂರು,6288,3.84,18689,19029,65214,102932
7,7,ೊಪಳ,3355,2.05,8591,4560,35964,49115
8,8,ಗದ,3971,2.42,9518,4286,48654,62458
9,9,ಾರಾಡ,3607,2.20,6698,4087,48535,59320


In [52]:
len(updated_list)

6

In [53]:
def reset_header(df_list):
    new_df_list = []

    for df in df_list:
        df.columns = df.iloc[0]
        df = df.drop(0)

        # Reset the index to ensure correct alignment
        df = df.reset_index(drop=True)
        new_df_list.append(df)
    return new_df_list

In [54]:
final_list = reset_header(updated_list)

In [55]:
final_list[0]

,1,2,1341,1342,1343,1344,1345,1346
0,1,ೆಳಾ,16023,9.78,40242,13039,202300,255581
1,2,ಾಗಲೋೆ,4879,2.98,15892,7282,49554,72728
2,3,ಜಯಪರ,4708,2.87,14027,830,39060,53917
3,4,ಕಲು,7655,4.67,30228,1326,76536,108090
4,5,ೕದ,4051,2.47,14588,5986,22034,42608
5,6,ಾಯಚೂರು,6288,3.84,18689,19029,65214,102932
6,7,ೊಪಳ,3355,2.05,8591,4560,35964,49115
7,8,ಗದ,3971,2.42,9518,4286,48654,62458
8,9,ಾರಾಡ,3607,2.20,6698,4087,48535,59320
9,10,ಉತರ ಕನಡ,2668,1.63,3864,1057,32827,37748


In [56]:
def get_empty_col(df_list):
    index_list=[]
    for i in range(len(df_list)):
        empty_columns = df_list[i].columns[df_list[i].isna().all()].tolist()
        if len(empty_columns) > 0:
            index_list.append([i,empty_columns])
    return index_list

In [57]:
empty_col = get_empty_col(final_list)
empty_col

[]

In [58]:
def concat_dataframe(df_list):
    df = pd.DataFrame()
    if len(df_list)>0:
        df = pd.concat([df, df_list[0].iloc[:,:2]], axis = 1)
        for item in df_list:
            df = pd.concat([df, item.iloc[:,2:]], axis = 1)
    return df

In [59]:
final_df = concat_dataframe(final_list)

In [60]:
final_df

,1,2,1341,1342,1343,1344,1345,1346,1347,1348,...,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375
0,1,ೆಳಾ,16023,9.78,40242,13039,202300,255581,153.5300,12533,...,478786,5190,4995,469938,400976,67855,1107,104,2600,0
1,2,ಾಗಲೋೆ,4879,2.98,15892,7282,49554,72728,67.9100,4463,...,214198,2221,2164,183513,156871,26261,381,23,575,168
2,3,ಜಯಪರ,4708,2.87,14027,830,39060,53917,933.7700,2309,...,261827,2313,2269,185957,188205,26640,202,64,1750,0
3,4,ಕಲು,7655,4.67,30228,1326,76536,108090,2.2931,0,...,250790,3130,3057,176588,149372,26576,640,0,0,175
4,5,ೕದ,4051,2.47,14588,5986,22034,42608,335.9400,3373,...,210970,1893,1866,137255,147956,39168,323,0,137255,150
5,6,ಾಯಚೂರು,6288,3.84,18689,19029,65214,102932,109.3100,5146,...,241913,2662,2621,193182,150963,41539,663,0,12,300
6,7,ೊಪಳ,3355,2.05,8591,4560,35964,49115,1.2512,37,...,162468,1804,1708,137452,103503,33256,693,0,0,100
7,8,ಗದ,3971,2.42,9518,4286,48654,62458,2.2795,21,...,95045,1136,1099,86100,59128,17969,473,20,500,25
8,9,ಾರಾಡ,3607,2.20,6698,4087,48535,59320,3.1700,32,...,146286,1507,1497,118331,99310,18848,176,0,0,83
9,10,ಉತರ ಕನಡ,2668,1.63,3864,1057,32827,37748,695.6700,54,...,105950,2453,2616,108918,90182,6395,290,28,683,0


In [61]:
final_df.columns = final_df.columns.astype(int)

In [62]:
final_df[2]

0            ೆಳಾ
1          ಾಗಲೋೆ
2            ಜಯಪರ
3            ಕಲು
4              ೕದ
5           ಾಯಚೂರು
6             ೊಪಳ
7               ಗದ
8            ಾರಾಡ
9          ಉತರ ಕನಡ
10            ಾೇ
11            ಬಾ
12          ತದುಗ
13         ಾವಣೆೆ
14             ವಗ
15             ಉಡು
16        ಕಮಗಳರು
17          ತುಮಕೂರು
18            ೋಾರ
19      ೆ೦ಗಳರು(ನ)
20    ೆ೦ಗಳರು(ಾ)
21             ಮಂಡ
22             ಾಸನ
23         ದಣ ಕನಡ
24            ೊಡಗು
25           ೖಸೂರು
26        ಾಮಾಜನಗರ
27       ಕಬಾಪರ
28           ಾಮನಗರ
29            ಾದ
Name: 2, dtype: object

In [63]:
final_df = final_df.reindex(sorted(final_df.columns, key=int), axis=1)

In [64]:
final_df

,1,2,1341,1342,1343,1344,1345,1346,1347,1348,...,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375
0,1,ೆಳಾ,16023,9.78,40242,13039,202300,255581,153.5300,12533,...,478786,5190,4995,469938,400976,67855,1107,104,2600,0
1,2,ಾಗಲೋೆ,4879,2.98,15892,7282,49554,72728,67.9100,4463,...,214198,2221,2164,183513,156871,26261,381,23,575,168
2,3,ಜಯಪರ,4708,2.87,14027,830,39060,53917,933.7700,2309,...,261827,2313,2269,185957,188205,26640,202,64,1750,0
3,4,ಕಲು,7655,4.67,30228,1326,76536,108090,2.2931,0,...,250790,3130,3057,176588,149372,26576,640,0,0,175
4,5,ೕದ,4051,2.47,14588,5986,22034,42608,335.9400,3373,...,210970,1893,1866,137255,147956,39168,323,0,137255,150
5,6,ಾಯಚೂರು,6288,3.84,18689,19029,65214,102932,109.3100,5146,...,241913,2662,2621,193182,150963,41539,663,0,12,300
6,7,ೊಪಳ,3355,2.05,8591,4560,35964,49115,1.2512,37,...,162468,1804,1708,137452,103503,33256,693,0,0,100
7,8,ಗದ,3971,2.42,9518,4286,48654,62458,2.2795,21,...,95045,1136,1099,86100,59128,17969,473,20,500,25
8,9,ಾರಾಡ,3607,2.20,6698,4087,48535,59320,3.1700,32,...,146286,1507,1497,118331,99310,18848,176,0,0,83
9,10,ಉತರ ಕನಡ,2668,1.63,3864,1057,32827,37748,695.6700,54,...,105950,2453,2616,108918,90182,6395,290,28,683,0


In [65]:
final_df = final_df.drop(final_df.index[30:]) 

In [66]:
final_df

,1,2,1341,1342,1343,1344,1345,1346,1347,1348,...,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375
0,1,ೆಳಾ,16023,9.78,40242,13039,202300,255581,153.5300,12533,...,478786,5190,4995,469938,400976,67855,1107,104,2600,0
1,2,ಾಗಲೋೆ,4879,2.98,15892,7282,49554,72728,67.9100,4463,...,214198,2221,2164,183513,156871,26261,381,23,575,168
2,3,ಜಯಪರ,4708,2.87,14027,830,39060,53917,933.7700,2309,...,261827,2313,2269,185957,188205,26640,202,64,1750,0
3,4,ಕಲು,7655,4.67,30228,1326,76536,108090,2.2931,0,...,250790,3130,3057,176588,149372,26576,640,0,0,175
4,5,ೕದ,4051,2.47,14588,5986,22034,42608,335.9400,3373,...,210970,1893,1866,137255,147956,39168,323,0,137255,150
5,6,ಾಯಚೂರು,6288,3.84,18689,19029,65214,102932,109.3100,5146,...,241913,2662,2621,193182,150963,41539,663,0,12,300
6,7,ೊಪಳ,3355,2.05,8591,4560,35964,49115,1.2512,37,...,162468,1804,1708,137452,103503,33256,693,0,0,100
7,8,ಗದ,3971,2.42,9518,4286,48654,62458,2.2795,21,...,95045,1136,1099,86100,59128,17969,473,20,500,25
8,9,ಾರಾಡ,3607,2.20,6698,4087,48535,59320,3.1700,32,...,146286,1507,1497,118331,99310,18848,176,0,0,83
9,10,ಉತರ ಕನಡ,2668,1.63,3864,1057,32827,37748,695.6700,54,...,105950,2453,2616,108918,90182,6395,290,28,683,0


In [67]:
new_order = [18, 20, 19, 27, 26, 29, 30, 22, 21, 24, 23, 25, 4, 5, 8, 12, 10, 9, 6, 1, 2, 15, 13, 11, 14, 16, 17, 7, 3, 28]
final_df[1] = new_order
final_df

,1,2,1341,1342,1343,1344,1345,1346,1347,1348,...,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375
0,18,ೆಳಾ,16023,9.78,40242,13039,202300,255581,153.5300,12533,...,478786,5190,4995,469938,400976,67855,1107,104,2600,0
1,20,ಾಗಲೋೆ,4879,2.98,15892,7282,49554,72728,67.9100,4463,...,214198,2221,2164,183513,156871,26261,381,23,575,168
2,19,ಜಯಪರ,4708,2.87,14027,830,39060,53917,933.7700,2309,...,261827,2313,2269,185957,188205,26640,202,64,1750,0
3,27,ಕಲು,7655,4.67,30228,1326,76536,108090,2.2931,0,...,250790,3130,3057,176588,149372,26576,640,0,0,175
4,26,ೕದ,4051,2.47,14588,5986,22034,42608,335.9400,3373,...,210970,1893,1866,137255,147956,39168,323,0,137255,150
5,29,ಾಯಚೂರು,6288,3.84,18689,19029,65214,102932,109.3100,5146,...,241913,2662,2621,193182,150963,41539,663,0,12,300
6,30,ೊಪಳ,3355,2.05,8591,4560,35964,49115,1.2512,37,...,162468,1804,1708,137452,103503,33256,693,0,0,100
7,22,ಗದ,3971,2.42,9518,4286,48654,62458,2.2795,21,...,95045,1136,1099,86100,59128,17969,473,20,500,25
8,21,ಾರಾಡ,3607,2.20,6698,4087,48535,59320,3.1700,32,...,146286,1507,1497,118331,99310,18848,176,0,0,83
9,24,ಉತರ ಕನಡ,2668,1.63,3864,1057,32827,37748,695.6700,54,...,105950,2453,2616,108918,90182,6395,290,28,683,0


In [68]:
final_data = final_df.sort_values(1)
final_data.reset_index(drop=True, inplace=True)
final_data

,1,2,1341,1342,1343,1344,1345,1346,1347,1348,...,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375
0,1,ೆ೦ಗಳರು(ನ),5554,3.39,23657,2495,80472,106624,1.8514,202,...,219204,2422,2329,174216,160687,13374,155,10,305,0
1,2,ೆ೦ಗಳರು(ಾ),3872,2.36,15577,4438,40995,61010,604.8400,25,...,67991,1230,1109,50024,48913,1085,26,0,0,68
2,3,ಾಮನಗರ,5086,3.10,22054,1740,61538,85332,110.5700,4619,...,64781,1543,1301,51604,48545,2878,181,0,0,248
3,4,ತದುಗ,6148,3.75,24476,17234,46880,88590,136.3500,5581,...,157219,2333,2217,125463,112146,13001,316,44,1075,282
4,5,ಾವಣೆೆ,5822,3.55,19762,12914,43817,76493,148.0100,5010,...,140712,1721,1656,119189,107368,11599,222,1,0,97
5,6,ೋಾರ,5009,3.06,22847,5001,46962,74810,33.5700,2969,...,108231,2061,1980,83498,80024,3370,104,14,0,25
6,7,ಕಬಾಪರ,5759,3.52,18858,11464,64563,94885,38.9456,4083,...,95323,1961,1848,78560,71847,6568,145,0,0,100
7,8,ವಗ,5000,3.05,19714,4472,52102,76288,91.2900,4828,...,103738,2377,2073,129237,119781,9182,274,29,725,155
8,9,ತುಮಕೂರು,11040,6.74,35103,13891,128083,177077,9.9900,152,...,154408,4011,3467,140700,128779,10186,158,2,50,325
9,10,ಕಮಗಳರು,3683,2.25,14271,2704,43372,60347,6.8600,107,...,62700,1825,1667,57236,53227,3787,222,10,250,25


In [69]:
name_list = ['Bengaluru(U)','Bengaluru(R)', 'Ramanagara', 'Chitradurga', 'Davanagere', 'Kolara', 'Chikkaballapura', 'Shivamogga',
'Tumakuru', 'Chikkamagaluru', 'Dakshina Kannada', 'Udupi', 'Hassan', 'Kodagu', 'Mandya', 'Mysuru', 'Chamarajanagara',
 'Belagavi', 'Vijayapura', 'Bagalkot', 'Dharwad', 'Gadag', 'Haveri', 'Uttara Kannada', 'Ballari', 'Bidar', 'Kalburgi',
 'Yadgir', 'Raichur', 'Koppal']

In [70]:
final_data[2] = name_list
final_data

,1,2,1341,1342,1343,1344,1345,1346,1347,1348,...,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375
0,1,Bengaluru(U),5554,3.39,23657,2495,80472,106624,1.8514,202,...,219204,2422,2329,174216,160687,13374,155,10,305,0
1,2,Bengaluru(R),3872,2.36,15577,4438,40995,61010,604.8400,25,...,67991,1230,1109,50024,48913,1085,26,0,0,68
2,3,Ramanagara,5086,3.10,22054,1740,61538,85332,110.5700,4619,...,64781,1543,1301,51604,48545,2878,181,0,0,248
3,4,Chitradurga,6148,3.75,24476,17234,46880,88590,136.3500,5581,...,157219,2333,2217,125463,112146,13001,316,44,1075,282
4,5,Davanagere,5822,3.55,19762,12914,43817,76493,148.0100,5010,...,140712,1721,1656,119189,107368,11599,222,1,0,97
5,6,Kolara,5009,3.06,22847,5001,46962,74810,33.5700,2969,...,108231,2061,1980,83498,80024,3370,104,14,0,25
6,7,Chikkaballapura,5759,3.52,18858,11464,64563,94885,38.9456,4083,...,95323,1961,1848,78560,71847,6568,145,0,0,100
7,8,Shivamogga,5000,3.05,19714,4472,52102,76288,91.2900,4828,...,103738,2377,2073,129237,119781,9182,274,29,725,155
8,9,Tumakuru,11040,6.74,35103,13891,128083,177077,9.9900,152,...,154408,4011,3467,140700,128779,10186,158,2,50,325
9,10,Chikkamagaluru,3683,2.25,14271,2704,43372,60347,6.8600,107,...,62700,1825,1667,57236,53227,3787,222,10,250,25


In [71]:
final_data.to_csv(absolute_result_path, sep=';', index=False)
final_data.to_csv(absolute_dictionary_path, sep=';', index=False)